$$\min c^T x \quad\text{subject to}\quad Ax\leq b $$
$$\max -b^T z \quad\text{subject to}\quad A^T z+c=0, z\geq 0.$$

In [139]:
import numpy as np

In [140]:
m = 100
n = 50
A = np.random.randn(m, n)
b = np.random.rand(m, )
z0 = np.random.rand(m, )
# To make the dual is feasible at the equality: A^T z + c == 0.
c = -A.T@z0
# linprog
from scipy.optimize import linprog
res = linprog(c, A_ub=A, b_ub=b, method='highs')
x = res.x
s = (1-c.T@x)/np.linalg.norm(c)**2
b = b + s * A @ c
x0 = s * c

In [141]:
MAXITERS = 500
TOL = 1e-8
RESTOL = 1e-8
MU = 10
ALPHA = 0.01
BETA = 0.5

[m, n] = A.shape
gaps = []
resdls = []
x = x0
# -h(x)
s = b - A @ x
# The z is initialized to be the inverse of -h(x) so the initial gap is 100(dimensions) * 1
z = 1 / s

for iters in range(MAXITERS):
    # Set the surrogate duality gap.
    gap = s.T @ z
    gaps.append(gap)
    # res_dual
    res = A.T @ z + c
    # res_prime is empty since there is no equality constraints in the primal problem.
    resdls.append(np.linalg.norm(res))

    if gap < TOL and np.linalg.norm(res) < RESTOL:
        break

    tinv = gap / (m * MU)
    sol = -np.linalg.solve(
        np.block([
                    [np.zeros((n, n)), A.T], 
                    [A, np.diag(-s / z)]
                 ]),
        np.concatenate((A.T @ z + c, -s + tinv * (1 / z)))
    )

    dx = sol[:n]
    dz = sol[n:(n + m)]
    ds = -A @ dx
    
    # backtracking line search
    step = min(1.0, 0.99 / np.max(-dz / z))

    while np.min(s + step * ds) <= 0:
        step = BETA * step

    newz = z + step * dz
    newx = x + step * dx
    news = s + step * ds
    newr = np.concatenate((c + A.T @ newz, newz * news - tinv))

    # Old residual.
    r = np.concatenate((c + A.T @ z, z * s - tinv))
    while np.linalg.norm(newr) > (1 - ALPHA * step) * np.linalg.norm(r):
        step = BETA * step
        newz = z + step * dz
        newx = x + step * dx
        news = s + step * ds
        newr = np.concatenate((c + A.T @ newz, newz * news - tinv))
        
    x = newx
    z = newz
    s = b - A @ x
    print('iter: {}, gap: {}, res_dual: {}'.format(iters, gap, np.linalg.norm(res)))


iter: 0, gap: 100.0, res_dual: 894.2484227434305
iter: 1, gap: 16.110453809592254, res_dual: 62.87045520059464
iter: 2, gap: 10.00560600303271, res_dual: 36.385843937668
iter: 3, gap: 4.685846439421022, res_dual: 15.21988448711682
iter: 4, gap: 2.75725402848967, res_dual: 8.395407707303848
iter: 5, gap: 2.36506071646642, res_dual: 7.089395797157947
iter: 6, gap: 1.9967558630547144, res_dual: 5.938385186614892
iter: 7, gap: 1.685513313931276, res_dual: 4.958607726885616
iter: 8, gap: 1.4902481378872692, res_dual: 4.335913532042667
iter: 9, gap: 1.0375154994287343, res_dual: 2.9397477073061653
iter: 10, gap: 0.7501242097581367, res_dual: 2.0567149578277704
iter: 11, gap: 0.3989155026969691, res_dual: 1.0270256606048613
iter: 12, gap: 0.23112306150126208, res_dual: 0.5671979693261456
iter: 13, gap: 0.18923855441964219, res_dual: 0.45503044366813566
iter: 14, gap: 0.121624248343835, res_dual: 0.27763031384888953
iter: 15, gap: 0.05544020652380569, res_dual: 0.11162515530876196
iter: 16, ga

In [142]:
from newton import newton

## Solve by Log-barrier method
In this case, we have the problem
$$\min t c^T x-\sum_{i=1}^m\log(b_i-a_i^T x)$$

In [143]:
mu=1000000
t = 10
x_inner = x0
while m/t > TOL:
    f = lambda x: t*c.dot(x) - np.sum(np.log(b-A@x))
    grad_f = lambda x: t*c - A.T @ (1/(b-A@x))
    nabla_f = lambda x: A.T @ np.diag(1/(b-A@x)**2) @ A

    x = newton(f, grad_f, nabla_f, x_inner, A, b, MAXITERS=50, TOL=1e-5, alpha=0.0001, beta=0.8)
    x_inner = x
    t *= mu

print('x: ', x)

x:  [-0.24206887  0.15233649 -0.20740402  0.04782789 -0.11393889  0.08104318
 -0.07055877  0.09576275 -0.04364514 -0.10902914 -0.02719174  0.06773663
 -0.13100569  0.01561854  0.08616531  0.07975069 -0.05951669  0.04148461
  0.01312011 -0.07625093 -0.1693231   0.04033728 -0.15916081  0.02789402
  0.11340647  0.07808953 -0.01352823 -0.0969958  -0.05271809  0.00858672
  0.09249804  0.0854395  -0.16170764 -0.13991566  0.12032338 -0.04428461
 -0.13360116 -0.00648328  0.02792178 -0.00762344 -0.00475129  0.09871693
  0.22407866 -0.04578106 -0.16553744  0.01162974  0.06919277  0.12544177
 -0.01070682 -0.01046722]


In [144]:
newx

array([-0.23604906,  0.12377964, -0.17639784,  0.03790132, -0.1062847 ,
        0.11685993, -0.09015006,  0.05889853, -0.03352306, -0.10809468,
       -0.03362739,  0.07375203, -0.11936696, -0.00976464,  0.08418517,
        0.09705917, -0.06667397,  0.02236559,  0.01670618, -0.09751127,
       -0.14900904,  0.04188163, -0.1648364 ,  0.04195258,  0.09487956,
        0.05482955,  0.00820284, -0.06430692, -0.03197153,  0.00627827,
        0.12504561,  0.09933419, -0.1618166 , -0.13074854,  0.14864624,
       -0.05135016, -0.12330055, -0.00047283,  0.0081143 , -0.03976628,
        0.00950227,  0.11194858,  0.15832168,  0.00500189, -0.17083775,
        0.01676946,  0.0771982 ,  0.10031471,  0.01732026,  0.00190627])